<a href="https://colab.research.google.com/github/jacobdwatters/NIOSH-Project/blob/main/SIG_SUB_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
import numpy as np
from numpy.random import seed
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, make_scorer, roc_auc_score
from sklearn.metrics import r2_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve, balanced_accuracy_score, accuracy_score, cohen_kappa_score, confusion_matrix, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import lightgbm as lgb
from skranger.ensemble import RangerForestClassifier

import pickle

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

import violation_common

from importlib import reload
reload(violation_common)

from pprint import pprint


import scikitplot as skplt

import scipy as sp
from scipy import stats

import joblib
from scipy.stats import chi2_contingency
import optuna

In [10]:
# Load data
train_full = pd.read_csv('data/after_2010_train_full.csv', index_col=0)
test = pd.read_csv('data/after_2010_test.csv', index_col=0)
train_hp = pd.read_csv('data/after_2010_train_hp.csv', index_col=0)
validate_hp = pd.read_csv('data/after_2010_validate_hp.csv', index_col=0)
train_smote_full = pd.read_csv('data/after_2010_train_smote_full.csv', index_col=0)
train_hp_smote = pd.read_csv('data/after_2010_train_smote_hp.csv', index_col=0)

categorical_cols = ['PRIMARY_OR_MILL', 'COAL_METAL_IND', 'MINE_TYPE', 'VIOLATOR_TYPE_CD']
numerical_cols = ['VIOLATOR_INSPECTION_DAY_CNT', 'VIOLATOR_VIOLATION_CNT', 'YEAR_OCCUR']

target = 'SIG_SUB'

feature_ranking = [
    'YEAR_OCCUR',
    'VIOLATOR_VIOLATION_CNT',
    'VIOLATOR_INSPECTION_DAY_CNT',
    'MINE_TYPE',
    'PRIMARY_OR_MILL',
    'COAL_METAL_IND',
    'VIOLATOR_TYPE_CD'
]

In [92]:
def lgb_objective(trial, data_train, data_validate, metrics, objective_metric):

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    num_features = trial.suggest_int('num_features', 1, len(categorical_cols) + len(numerical_cols))

    X_train, y_train, preprocessor, target_transformer = violation_common.encode_and_scale(data_train, target=target, contionous_target=False, to_keep=feature_ranking[0:num_features], categorical_cols=categorical_cols, numerical_cols=numerical_cols, preprocessor=None, target_transformer=None)
    X_validate, y_validate, _, _ = violation_common.encode_and_scale(data_validate, target=target, to_keep=feature_ranking[0:num_features], preprocessor=preprocessor, target_transformer=target_transformer)


    model = lgb.LGBMClassifier(**param)
    


    model.fit(X_train, y_train)
    preds = model.predict_proba(X_validate)[:, 1]
    pred_labels = model.predict(X_validate)


    # calculate metrics
    for metric_name, metric_func, requires_proba in metrics:
        metric_value = None
        if requires_proba:
            metric_value = metric_func(y_validate, preds)
        else:
            metric_value = metric_func(y_validate, pred_labels)
        trial.set_user_attr(metric_name, metric_value)

    return trial.user_attrs[objective_metric]

In [108]:
def rf_objective(trial, data_train, data_validate, metrics, objective_metric):
    num_features = trial.suggest_int('num_features', 1, len(categorical_cols) + len(numerical_cols))
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 64, 128),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "min_node_size": trial.suggest_int("min_node_size", 1, 5),
        "mtry": trial.suggest_int("mtry", 0, num_features),  # num of features
        "split_rule": trial.suggest_categorical("split_rule", ["gini", "extratrees"]),
        "sample_fraction": trial.suggest_float("sample_fraction", 0.5, 1.0),
    }

    X_train, y_train, preprocessor, target_transformer = violation_common.encode_and_scale(data_train, target=target, contionous_target=False, to_keep=feature_ranking[0:num_features], categorical_cols=categorical_cols, numerical_cols=numerical_cols, preprocessor=None, target_transformer=None)
    X_validate, y_validate, _, _ = violation_common.encode_and_scale(data_validate, target=target, to_keep=feature_ranking[0:num_features], preprocessor=preprocessor, target_transformer=target_transformer)

    model = RangerForestClassifier(**param)

    model.fit(X_train, y_train)
    preds = model.predict_proba(X_validate)[:, 1]
    pred_labels = model.predict(X_validate)


    # calculate metrics
    for metric_name, metric_func, requires_proba in metrics:
        metric_value = None
        if requires_proba:
            metric_value = metric_func(y_validate, preds)
        else:
            metric_value = metric_func(y_validate, pred_labels)
        trial.set_user_attr(metric_name, metric_value)

    return trial.user_attrs[objective_metric]

In [111]:
def logistic_objective(trial, data_train, data_validate, metrics, objective_metric):
    num_features = trial.suggest_int('num_features', 1, len(categorical_cols) + len(numerical_cols))
    param = {
        "C": trial.suggest_float("C", 1e-10, 1e10, log=True),
        "penalty": trial.suggest_categorical("penalty", ["l1", "l2"]),
        "solver": trial.suggest_categorical("solver", ["liblinear", "saga"]),
        "max_iter": trial.suggest_int("max_iter", 100, 1000),
    }

    X_train, y_train, preprocessor, target_transformer = violation_common.encode_and_scale(data_train, target=target, contionous_target=False, to_keep=feature_ranking[0:num_features], categorical_cols=categorical_cols, numerical_cols=numerical_cols, preprocessor=None, target_transformer=None)
    X_validate, y_validate, _, _ = violation_common.encode_and_scale(data_validate, target=target, to_keep=feature_ranking[0:num_features], preprocessor=preprocessor, target_transformer=target_transformer)

    model = LogisticRegression(**param)

    model.fit(X_train, y_train)
    preds = model.predict_proba(X_validate)[:, 1]
    pred_labels = model.predict(X_validate)


    # calculate metrics
    for metric_name, metric_func, requires_proba in metrics:
        metric_value = None
        if requires_proba:
            metric_value = metric_func(y_validate, preds)
        else:
            metric_value = metric_func(y_validate, pred_labels)
        trial.set_user_attr(metric_name, metric_value)

    return trial.user_attrs[objective_metric]

In [113]:
# Define custom dataset


class ClassifierDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float), torch.unsqueeze(torch.tensor(self.labels[idx], dtype=torch.float), dim=0)

def nn_objective(trial, data_train, data_validate, metrics, objective_metric):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Define neural network architecture
    num_features = trial.suggest_int('num_features', 1, len(categorical_cols) + len(numerical_cols))

    X_train, y_train, preprocessor, target_transformer = violation_common.encode_and_scale(data_train, target=target, contionous_target=False, to_keep=feature_ranking[0:num_features], categorical_cols=categorical_cols, numerical_cols=numerical_cols, preprocessor=None, target_transformer=None)
    X_validate, y_validate, _, _ = violation_common.encode_and_scale(data_validate, target=target, to_keep=feature_ranking[0:num_features], preprocessor=preprocessor, target_transformer=target_transformer)


    input_size = X_train.shape[1]
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)

    n_layers = trial.suggest_int('n_layers', 1, 3)
    layers = []
    for i in range(n_layers):
        if i == 0:
            layers.append(nn.Linear(input_size, trial.suggest_int(f'n_units_l{i}', 10, 500)))
        else:
            layers.append(nn.Linear(trial.suggest_int(f'n_units_l{i-1}', 10, 500), 
                                    trial.suggest_int(f'n_units_l{i}', 10, 500)))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_rate))
    layers.append(nn.Linear(trial.suggest_int(f'n_units_l{n_layers-1}', 10, 500), 1))
    layers.append(nn.Sigmoid())

    model = nn.Sequential(*layers).to(device)

    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()
    
    model_train_x, early_stopping_x, model_train_y, early_stopping_y = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


    # Create data loaders
    model_train_dataset = ClassifierDataset(model_train_x, model_train_y)
    val_dataset = ClassifierDataset(X_validate, y_validate)
    early_stopping_dataset = ClassifierDataset(early_stopping_x, early_stopping_y)
    train_loader = DataLoader(model_train_dataset, batch_size=1024, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    early_stopping_loader = DataLoader(early_stopping_dataset, batch_size=32, shuffle=False)

    early_stopping_best = 0
    early_stopping_strikes = 0

    # Training loop
    for epoch in range(100):
        model.train()
        for batch in tqdm(train_loader):
            features, labels = batch
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        # early stopping
        model.eval()
        with torch.no_grad():
            preds = []
            targets = []
            for batch in early_stopping_loader:
                features, labels = batch
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                preds.extend(outputs.cpu().numpy())
                targets.extend(labels.cpu().numpy())
            
        early_stop_auc = roc_auc_score(targets, preds)
        print(f'Early stopping auc: {early_stop_auc}')
        
        if early_stop_auc > early_stopping_best + 0.01:
            early_stopping_best = early_stop_auc
            early_stopping_strikes = 0
        else:
            if early_stop_auc > early_stopping_best:
                early_stopping_best = early_stop_auc
            early_stopping_strikes += 1
            if early_stopping_strikes == 2:
                break

    # Validation
    model.eval()
    preds = []
    targets = []
    with torch.no_grad():
        for batch in val_loader:
            features, labels = batch
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            preds.extend(outputs.cpu().numpy())
            targets.extend(labels.cpu().numpy())

    pred_labels = np.array(preds) > 0.5
        
    # calculate metrics
    for metric_name, metric_func, requires_proba in metrics:
        metric_value = None
        if requires_proba:
            metric_value = metric_func(y_validate, preds)
        else:
            metric_value = metric_func(y_validate, pred_labels)
        trial.set_user_attr(metric_name, metric_value)

    return trial.user_attrs[objective_metric]

In [ ]:
dataset_types = ['non_smote', 'smote']
#model_types = [('lightgbm', lgb_objective)]
#model_types = [('random_forest', rf_objective)]
#model_types = [('logistic_regression', logistic_objective)]
#model_types = [('neural_network', nn_objective)]

# model types is a list of tuples of model names and objective functions and number of trials
model_types = [('lightgbm', lgb_objective, 10), ('random_forest', rf_objective, 10), ('logistic_regression', logistic_objective, 10), ('neural_network', nn_objective, 5)]

# list of tuples of metric names and functions along with whether they require a probability prediction
metrics = [('roc_auc_score', roc_auc_score, True),
           ('accuracy_score', accuracy_score, False),
           ('balanced_accuracy_score', balanced_accuracy_score, False),
           ('f1_score', f1_score, False),
           ('precision_score', precision_score, False),
           ('recall_score', recall_score, False),
           ('cohen_kappa_score', cohen_kappa_score, False),
           ('confusion_matrix', confusion_matrix, False)]

# results[dataset_type][model_name] = study
hp_validation_results = {dataset_type: {model_name[0]: None for model_name in model_types} for dataset_type in dataset_types}

for dataset_type in dataset_types:
    for model_name, objective, n_trials in model_types:
        print(f'SMOTE: {dataset_type}, Model: {model_name}')
        study = optuna.create_study(direction="maximize")
        trial_train_data = train_hp_smote if dataset_type == 'smote' else train_hp
        study.optimize(lambda trial: objective(trial,
                                               data_train=trial_train_data,
                                               data_validate=validate_hp,
                                               metrics=metrics,
                                               objective_metric='roc_auc_score'),
                       n_trials=n_trials)

        print("Best trial:")
        trial = study.best_trial

        print("  Value: {}".format(trial.value))

        print("  Params: ")
        for key, value in trial.params.items():
            print("    {}: {}".format(key, value))
            
        hp_validation_results[dataset_type][model_name] = study

In [123]:
for dataset_type in dataset_types:
    for model_name, objective, _ in model_types:
        print(f'SMOTE: {dataset_type}, Model: {model_name}')
        print("Best hyperparameters:")
        pprint(hp_validation_results[dataset_type][model_name].best_params)
        print("Metrics:")
        pprint(hp_validation_results[dataset_type][model_name].best_trial.user_attrs)
        print()

# Save results with pickle
with open('data/hp_validation_results.pkl', 'wb') as f:
    pickle.dump(hp_validation_results, f)

SMOTE: non_smote, Model: lightgbm
Best hyperparameters:
{'bagging_fraction': 0.578853457712694,
 'bagging_freq': 2,
 'feature_fraction': 0.9432804076931858,
 'lambda_l1': 0.00022058704416932193,
 'lambda_l2': 0.8186572930303238,
 'min_child_samples': 31,
 'num_features': 5,
 'num_leaves': 171}
Metrics:
{'accuracy_score': 0.767914537763406,
 'balanced_accuracy_score': 0.502116592144414,
 'cohen_kappa_score': 0.006463502879039296,
 'confusion_matrix': array([[156939,    227],
       [ 47286,    270]]),
 'f1_score': 0.011237591825692464,
 'precision_score': 0.5432595573440644,
 'recall_score': 0.005677517032551098,
 'roc_auc_score': 0.5914903380005341}

SMOTE: non_smote, Model: random_forest
Best hyperparameters:
{'max_depth': 9,
 'min_node_size': 2,
 'mtry': 5,
 'n_estimators': 108,
 'num_features': 5,
 'sample_fraction': 0.7689132514325179,
 'split_rule': 'gini'}
Metrics:
{'accuracy_score': 0.7677777669229492,
 'balanced_accuracy_score': 0.5003630207881562,
 'cohen_kappa_score': 0.00111

In [120]:
with open('data/hp_validation_results.pkl', 'rb') as f:
    hp_validation_results = pickle.load(f)
    print(hp_validation_results['non_smote']['lightgbm'].best_params)

{'lambda_l1': 1.2683782903367314e-07, 'lambda_l2': 0.0008330722610290878, 'num_leaves': 187, 'feature_fraction': 0.8097281553546072, 'bagging_fraction': 0.6896292970533484, 'bagging_freq': 5, 'min_child_samples': 100, 'num_features': 6}
